## **Assignment- Week 5**
**Instructions:** 
- Answer each question on a separate cell, either in a markdown or code cell depending on the question.
- FOr e
- Use comments in your code

## **Dataset Description**
You are presented with a customer booking dataset that contains information about airline bookings, including the number of passengers, sales channel, trip type, purchase lead time, length of stay, flight hour, flight day, route, booking origin, and various flags indicating if the customer wanted extra baggage, preferred seat, or in-flight meals. The dataset also includes the total flight duration and a flag indicating if the booking was completed. 

**Using the data, provide the following answers:**
1. What is the shape of our dataframe and explain what the shape means?
2. How many columns have null values and how many duplicate records are in the dataset?
3. Remove duplicates, rows with null values and the `purchase lead` and `booking origin` columns, what is the new shape of the dataframe?
4. What are the different datatypes we have in the dataset?
5. What are the different `trip types` available to customers and which one is mostly used?
6. What is the most used `sales channel` for booking by customers?
7. Which day had the highest `total number of flights`?
8. How many customers wanted a `preferred seat` and in `flight meals`?
9. Which of the `trip type` had the highest average `flight duration`?
10. Which day of the week had the highest average `flight duration`?

## **Columns description**
**Note:**\
1 = True/Yes\
0 = False/No
- `num_passengers` = number of passengers travelling
- `sales_channel` = sales channel booking was made on
- `trip_type = trip` Type (Round Trip, One Way, Circle Trip)
- `purchase_lead` = number of days between travel date and booking date
- `length_of_stay` = number of days spent at destination
- `flight_hour` = hour (time) of flight departure
- `flight_day` = day of week of flight departure
- `booking_origin` = country from where booking was made
- `wants_extra_baggage` = if the customer wanted extra baggage in the booking
- `wants_preferred_seat` = if the customer wanted a preferred seat in the booking
- `wants_in_flight_meals` = if the customer wanted in-flight meals in the booking
- `flight_duration` = total duration of flight (in hours)
- `booking_complete` = flag indicating if the customer completed the booking

In [190]:
import pandas as pd
import numpy as np


In [191]:
df = pd.read_csv('customer_bookings.csv')


In [192]:
df.head()


,num_passengers,sales_channel,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
0,2,Internet,RoundTrip,262.0,19.0,7.0,Sat,AKLDEL,New Zealand,1.0,0.0,0.0,5.52,0
1,1,Internet,RoundTrip,112.0,20.0,3.0,Sat,AKLDEL,New Zealand,0.0,0.0,0.0,5.52,0
2,2,Internet,RoundTrip,243.0,22.0,17.0,Wed,AKLDEL,India,1.0,1.0,0.0,5.52,0
3,1,Internet,RoundTrip,96.0,31.0,4.0,Sat,AKLDEL,New Zealand,0.0,0.0,1.0,5.52,0
4,2,Internet,RoundTrip,68.0,22.0,15.0,Wed,AKLDEL,India,1.0,0.0,1.0,5.52,0


In [193]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50017 entries, 0 to 50016
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   num_passengers         50017 non-null  int64  
 1   sales_channel          50007 non-null  object 
 2   trip_type              50012 non-null  object 
 3   purchase_lead          49992 non-null  float64
 4   length_of_stay         50010 non-null  float64
 5   flight_hour            50002 non-null  float64
 6   flight_day             50013 non-null  object 
 7   route                  50014 non-null  object 
 8   booking_origin         50006 non-null  object 
 9   wants_extra_baggage    50013 non-null  float64
 10  wants_preferred_seat   50012 non-null  float64
 11  wants_in_flight_meals  50012 non-null  float64
 12  flight_duration        50004 non-null  float64
 13  booking_complete       50017 non-null  int64  
dtypes: float64(7), int64(2), object(5)
memory usage: 5.3+ 

# data cleaning


In [194]:
 #Checking for missing values
df.isnull().sum()

num_passengers            0
sales_channel            10
trip_type                 5
purchase_lead            25
length_of_stay            7
flight_hour              15
flight_day                4
route                     3
booking_origin           11
wants_extra_baggage       4
wants_preferred_seat      5
wants_in_flight_meals     5
flight_duration          13
booking_complete          0
dtype: int64

In [195]:
#dropping all colums with rows that have any missing values.
df.dropna(inplace=True)


In [196]:
#rechecking for missing values
df.isnull().sum()

num_passengers           0
sales_channel            0
trip_type                0
purchase_lead            0
length_of_stay           0
flight_hour              0
flight_day               0
route                    0
booking_origin           0
wants_extra_baggage      0
wants_preferred_seat     0
wants_in_flight_meals    0
flight_duration          0
booking_complete         0
dtype: int64

In [197]:
#Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# handling outliers

In [198]:
def print_bounds(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    print(f"{column} -> Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")


print_bounds(df, 'purchase_lead')

purchase_lead -> Lower Bound: -120.0, Upper Bound: 256.0


In [199]:
outliers = detect_outliers_iqr(df, 'purchase_lead')
print(f'Number of outliers in purchase_lead: {len(outliers)}')


Number of outliers in purchase_lead: 3449


In [200]:
df['purchase_lead'] = df['purchase_lead'].clip(lower=lower_bound, upper=upper_bound)


In [201]:
Q1 = df['flight_duration'].quantile(0.25)
Q3 = df['flight_duration'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Lower Bound: {lower_bound}")
print(f"Upper Bound: {upper_bound}")


Lower Bound: 0.8050000000000006
Upper Bound: 13.645


In [202]:
outliers = df[(df['flight_duration'] < lower_bound) | (df['flight_duration'] > upper_bound)]
print(f"Number of outliers in flight_duration: {len(outliers)}")
outliers


Number of outliers in flight_duration: 0


,num_passengers,sales_channel,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete


flight_duration: 0 means that values are within a normal and expected range.

# EXPLORATORY DATA ANALYSIS

**Using the data, provide the following answers:**
1. What is the shape of our dataframe and explain what the shape means?
2. How many columns have null values and how many duplicate records are in the dataset?
3. Remove duplicates, rows with null values and the `purchase lead` and `booking origin` columns, what is the new shape of the dataframe?
4. What are the different datatypes we have in the dataset?
5. What are the different `trip types` available to customers and which one is mostly used?
6. What is the most used `sales channel` for booking by customers?
7. Which day had the highest `total number of flights`?
8. How many customers wanted a `preferred seat` and in `flight meals`?
9. Which of the `trip type` had the highest average `flight duration`?
10. Which day of the week had the highest average `flight duration`?

In [203]:
#1. What is the shape of our dataframe and explain what the shape means?

print("Shape of the DataFrame:", df.shape)


Shape of the DataFrame: (49942, 14)


it means that the shape is a tuple having 49207 rows and 14 columns.


#2. How many columns have null values and how many duplicate records are in the dataset?


Number of columns with null values: 0
I handled all missing values during the data cleaning process.

In [204]:
# for duplicate records are in this dataset


duplicate_rows = df.duplicated().sum()
print("Number of duplicate records:", duplicate_rows)


Number of duplicate records: 889



The dataset contains 889 duplicate records

3. Remove duplicates, rows with null values and the `purchase lead` and `booking origin` columns, what is the new shape of the dataframe?

In [205]:

#to remove duplicate rows
df = df.drop_duplicates() 

#to remove rows with null values
df = df.dropna() 



#to drop the columns purchase lead and booking origin
columns_to_drop = ['purchase lead', 'booking origin']


existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

df = df.drop(columns=existing_columns_to_drop) 


In [206]:
#the new shape of the DataFrame
new_shape = df.shape
print(new_shape)


(49053, 14)


The new shape of the DataFrame is (49053, 14), meaning it now has 49,053 rows and 14 columns after removing duplicates, rows with null values, and the purchase lead and booking origin columns. 

In [207]:
#4. What are the different datatypes we have in the dataset?

data_types = df.dtypes
print(data_types)


num_passengers             int64
sales_channel             object
trip_type                 object
purchase_lead            float64
length_of_stay           float64
flight_hour              float64
flight_day                object
route                     object
booking_origin            object
wants_extra_baggage      float64
wants_preferred_seat     float64
wants_in_flight_meals    float64
flight_duration          float64
booking_complete           int64
dtype: object


The dataset contains the following data types:

1. int64:

   -num_passengers
   -booking_complete

2. float64:

   -purchase_lead
   -length_of_stay
   -flight_hour
   -wants_extra_baggage
   -wants_preferred_seat
   -wants_in_flight_meals
   -flight_duration

3. object:

   -sales_channel
   -trip_type
   -flight_day
   -route
   -booking_origin

In [208]:
#5. What are the different `trip types` available to customers and which one is mostly used?

trip_types = df['trip_type'].value_counts()
print(trip_types)


trip_type
RoundTrip     48552
OneWay          385
CircleTrip      116
Name: count, dtype: int64


The most commonly used trip type is RoundTrip with 48552 occurances.

In [209]:
#6. What is the most used `sales channel` for booking by customers?


sales_channel_usage = df['sales_channel'].value_counts()
print(sales_channel_usage)

sales_channel
Internet    43706
Mobile       5347
Name: count, dtype: int64


The most used sales channel for booking by customers is the Internet which has 43,706 occurrences. Followed by Mobile with 5,347 occurrences.

In [210]:
#7. Which day had the highest `total number of flights`?


#Step 1- Group by 'flight_day' and count occurrences
flights_by_day = df['flight_day'].value_counts()


#Step 2- Finding the day with the highest number of flights and counting as well for the day
most_flights_day = flights_by_day.idxmax() 
most_flights_count = flights_by_day.max() 


#Step 3- Printing the result
print("The day with the highest total number of flights is:", most_flights_day)
print("Total number of flights on that day:", most_flights_count)


The day with the highest total number of flights is: Mon
Total number of flights on that day: 7955


The day with the highest total number of flights is: Mon
Total number of flights on that day: 7955

In [211]:
#8. How many customers wanted a `preferred seat` and in `flight meals`?


#Counting how many customers wanted a preferred seat.
preferred_seat_count = df['wants_preferred_seat'].sum()

#Counting how many customers wanted in-flight meals
in_flight_meals_count = df['wants_in_flight_meals'].sum()

#to print the results
print("Number of customers who wanted a preferred seat:", preferred_seat_count)
print("Number of customers who wanted in-flight meals:", in_flight_meals_count)


Number of customers who wanted a preferred seat: 14521.0
Number of customers who wanted in-flight meals: 20952.0


The number of customers who wanted a:
* **Preferred seat**: 14,521
* **In-flight meals**: 20,952



In [212]:
#9. Which of the `trip type` had the highest average `flight duration`?


#Grouping the data by 'trip_type' then calculating the average 'flight_duration'
average_flight_duration = df.groupby('trip_type')['flight_duration'].mean()

#Finding the trip type with the highest average flight duration
highest_avg_trip_type = average_flight_duration.idxmax()
highest_avg_duration = average_flight_duration.max()

#to print the result
print("The trip type with the highest average flight duration is:", highest_avg_trip_type)
print("The highest average flight duration for this trip type is:", highest_avg_duration)


The trip type with the highest average flight duration is: OneWay
The highest average flight duration for this trip type is: 7.970623376623377


The `OneWay` trip type has the highest average flight duration, with an average of 7.970623376623377 approximately **7.97 hours**.


#10. Which day of the week had the highest average `flight duration`?

In [213]:
#Checking the unique values in the 'flight_day' to know what format it is
unique_flight_days = df['flight_day'].unique()

# Print the unique flight days
print(unique_flight_days)


['Sat' 'Wed' 'Thu' 'Mon' 'Sun' 'Tue' 'Fri']


In [214]:
#Checking for missing or null values in the 'flight_day' column
missing_flight_days = df['flight_day'].isnull().sum()

print(f"Number of missing 'flight_day' values: {missing_flight_days}")


Number of missing 'flight_day' values: 0


In [215]:
#Droping all rows where 'flight_day' is missing
df_cleaned = df.dropna(subset=['flight_day'])


In [216]:
# Check the unique 'flight_day' values again
unique_flight_days = df_cleaned['flight_day'].unique()
print(unique_flight_days)


['Sat' 'Wed' 'Thu' 'Mon' 'Sun' 'Tue' 'Fri']


In [217]:
# Group by 'flight_day' and calculate the average 'flight_duration'
avg_flight_duration_by_day = df_cleaned.groupby('flight_day')['flight_duration'].mean()

# Find the day with the highest average flight duration
highest_avg_flight_day = avg_flight_duration_by_day.idxmax()
highest_avg_flight_duration = avg_flight_duration_by_day.max()

# Print the result
print("The day of the week with the highest average flight duration is:", highest_avg_flight_day)
print("The highest average flight duration for this day is:", highest_avg_flight_duration)


The day of the week with the highest average flight duration is: Sun
The highest average flight duration for this day is: 7.379128604832424


 **Sunday** has the highest average flight duration, with an average of 7.379128604832424 approx **7.38 hours**.
